In [1]:
import re
import time
import pandas as pd
import json
import requests
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime

In [2]:
#한국투자 API 보안 인증키 발급 
APP_KEY = "PSDbkNNrIX3j60cW2MEFnBGZDZ1BzpgDxvC0"
APP_SECRET = "QfaZRX24K15ojO4yjG5o5dM+SUvJT4PYiK+tKKzgikyhmfrnTXoxOwn9rCf2hzXdtOhh4v3/O4bSFfQgS+u6ojdb6KgWuyub4GRraoWYaJTJyHSRqWUPab2GiCvrI/aIjx5S925MohrETq0k/eopFHgMInG0lmZdAh5TaegaZ1NDpNvPW8Y="
URL_BASE = "https://openapi.koreainvestment.com:9443"
PATH = "oauth2/tokenP"
URL = f"{URL_BASE}/{PATH}"

headers = {"content-type":"application/json"}
body = {"grant_type":"client_credentials",
        "appkey":APP_KEY, 
        "appsecret":APP_SECRET}

res = requests.post(URL, headers=headers, data=json.dumps(body))
ACCESS_TOKEN = res.json()["access_token"]

In [138]:
#당일 시가 정보 불러오기
def find_gap(stock_code):
        time.sleep(0.1)
        URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-time-itemchartprice'

        headers = {"Content-Type":"application/json; charset=utf-8", 
                "authorization": f"Bearer {ACCESS_TOKEN}",
                "appKey":APP_KEY,
                "appSecret":APP_SECRET,
                "tr_id":"FHPST01060000"
                }

        params =  {
                "fid_cond_mrkt_div_code": "J",
                "fid_input_hour_1": "090030",
                "fid_input_iscd": stock_code
        }

        res = requests.get(URL, headers=headers, params=params)
        return res

In [39]:
#주가 기본정보 불러오기
Kospi_df = pd.read_csv('data/kospi_code.csv')
Kosdaq_df = pd.read_csv('data/kosdaq_code.csv')
#코스피 분류
kospi = Kospi_df.filter(items= ['단축코드','한글명'])
kospi['class'] = 'kospi'
#코스닥 분류
kosdaq = Kosdaq_df.filter(items= ['단축코드','한글종목명'])
kosdaq['class'] = 'kospi'
#주가 종목코드 불러오기
kospi = kospi.rename(columns={'단축코드':'stock_code','한글명' : 'stock_name'})
kosdaq = kosdaq.rename(columns={'단축코드':'stock_code','한글종목명' : 'stock_name'})
kospi_kosdaq = pd.concat([kospi,kosdaq])

In [44]:
kospi_kosdaq = kospi_kosdaq.reset_index(drop=True)

In [58]:
kospi_kosdaq

,stock_code,stock_name,class
0,F70101786,벨기에코어오피스부동산(A),kospi
1,F70101B95,밀라노부동산,kospi
2,F70101B96,벨기에코어오피스2호,kospi
3,F70101B99,한국투자도쿄한조몬오피스,kospi
4,F70101B9A,한국투자뉴욕오피스1호,kospi
...,...,...,...
3693,900280,골든센츄리,kospi
3694,900120,씨케이에이치,kospi
3695,900250,크리스탈신소재,kospi
3696,900070,글로벌에스엠,kospi


In [139]:
#개장 직후 종목 검색
data = pd.DataFrame()
for i in range(len(kospi_kosdaq)):
    stock_code = kospi_kosdaq.stock_code[i]
    stock_name = kospi_kosdaq.stock_name[i]
    res = find_gap(stock_code)
    try : 
        gap_per = res.json().get('output2')[-1].get('prdy_ctrt')
        gap_hour = res.json().get('output2')[-1].get('stck_cntg_hour')
        temp = pd.DataFrame({
                        'stock_code': stock_code,
                        'stock_name': stock_name,
                        'gap_per': gap_per,
                        'gap_hour': gap_hour,
                        }, index=[0])
        data = pd.concat([data,temp],axis=0)
    except :
        pass

  

In [146]:
data = data.reset_index(drop=True)

In [150]:
data

,stock_code,stock_name,gap_per,gap_hour
0,F74601669,메리츠베트남주식혼합,0.00,090030
1,000020,동화약품,0.11,090026
2,000040,KR모터스,0.00,090002
3,000050,경방,0.09,090030
4,000070,삼양홀딩스,-0.38,090030
...,...,...,...,...
3088,900280,골든센츄리,1.30,090232
3089,900120,씨케이에이치,0.00,090013
3090,900250,크리스탈신소재,-1.01,090006
3091,900070,글로벌에스엠,-0.72,090021


In [153]:
data['gap_int'] = data.gap_per.apply(lambda x: float(x))

In [158]:
stock2 = data.query('gap_int >= 2.0').reset_index(drop=True)

In [159]:
stock2

,stock_code,stock_name,gap_per,gap_hour,gap_int
0,001390,KG케미칼,2.98,090422,2.98
1,001470,삼부토건,3.65,090526,3.65
2,001515,SK증권우,2.10,090030,2.10
3,001570,금양,2.94,090527,2.94
4,002020,코오롱,3.02,090412,3.02
...,...,...,...,...,...
218,396270,넥스트칩,2.47,090504,2.47
219,408920,에스케이증권7호스팩,3.64,090504,3.64
220,417180,핑거스토리,5.33,090526,5.33
221,417840,저스템,6.96,090511,6.96


In [160]:
stock2.to_csv('data/230228.csv', index=False, encoding='utf-8-sig')

In [161]:
URL = 'https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice'

headers = {"Content-Type":"application/json", 
        "authorization": f"Bearer {ACCESS_TOKEN}",
        "appKey":APP_KEY,
        "appSecret":APP_SECRET,
        "tr_id":"FHKST01010900"
        }

params = {
    "FID_COND_MRKT_DIV_CODE":"J",
    "FID_INPUT_ISCD":'001390',
}

res = requests.get(URL, headers=headers, params=params)   
res = res.json()

In [173]:
res.get('output')[0:10]

[{'stck_bsop_date': '20230228',
  'stck_clpr': '24250',
  'prdy_vrss': '750',
  'prdy_vrss_sign': '2',
  'prsn_ntby_qty': '-30492',
  'frgn_ntby_qty': '26788',
  'orgn_ntby_qty': '680',
  'prsn_ntby_tr_pbmn': '-743',
  'frgn_ntby_tr_pbmn': '653',
  'orgn_ntby_tr_pbmn': '17',
  'prsn_shnu_vol': '73070',
  'frgn_shnu_vol': '41398',
  'orgn_shnu_vol': '798',
  'prsn_shnu_tr_pbmn': '1769',
  'frgn_shnu_tr_pbmn': '1006',
  'orgn_shnu_tr_pbmn': '19',
  'prsn_seln_vol': '103562',
  'frgn_seln_vol': '14610',
  'orgn_seln_vol': '118',
  'prsn_seln_tr_pbmn': '2512',
  'frgn_seln_tr_pbmn': '353',
  'orgn_seln_tr_pbmn': '3'},
 {'stck_bsop_date': '20230227',
  'stck_clpr': '23500',
  'prdy_vrss': '-100',
  'prdy_vrss_sign': '5',
  'prsn_ntby_qty': '-3603',
  'frgn_ntby_qty': '892',
  'orgn_ntby_qty': '-289',
  'prsn_ntby_tr_pbmn': '-84',
  'frgn_ntby_tr_pbmn': '20',
  'orgn_ntby_tr_pbmn': '-7',
  'prsn_shnu_vol': '41221',
  'frgn_shnu_vol': '6819',
  'orgn_shnu_vol': '139',
  'prsn_shnu_tr_pbmn': '